In [ ]:
import torch
import polyak


x = torch.randn(2, requires_grad=True, dtype=torch.double)
def loss_function():
    return sum(torch.abs(x))

def closure():
    optimizer.zero_grad()
    loss = loss_function()
    loss.backward()
    return loss

param = [x]
optimizer = polyak.Polyak(param)
# A short loop that applies the Polyak subgradient method to the loss function
for i in range(100):
    loss = optimizer.step(closure)
    print("Iteration: {}, Loss: {}".format(i, loss))
    print("x: {}".format(x.data))


In [ ]:
# Quick working test of superpolyak with numpy functions
import util
import numpy as np
import torch

def f(y):
    return np.sum(np.abs(y))

def gradf(y):
    return np.sign(y)

d = 5
max_elts = d
y0 = np.random.randn(d)
while f(y0) > 1e-20:
    y, d = util.build_bundle(f=f, gradf=gradf, y0=y0, tau=1.,eta_est=1.5,min_f=0., max_elts=max_elts)
    y0 = y
    print("f(y)", f(y0))

In [ ]:
# A pytorch compatible version of the above code.
import util
import numpy as np
import torch

d = 5
max_elts = d
x = torch.randn(d, requires_grad=True, dtype=torch.double)
# x.data = x.data / torch.norm(x.data)
def f():
    return torch.sum(abs(x))

# Closure function to allow us to call backward.
def closure():
    if x.grad is None:
        x.grad = torch.zeros_like(x)
    else:
        x.grad.data.zero_()
    loss = f()
    loss.backward()
    return loss

current_iter = 0
while closure().item() > 1e-20 and current_iter < 100:
    y, bundle_idx = util.build_bundle_torch(closure, x=x, tau=np.inf,eta_est=1.5,min_f=0., max_elts=max_elts)
    x.data = torch.Tensor(y)
    print("f(y)", closure().item())
    print("Bundle index", bundle_idx)
    current_iter += 1